In [2]:
#installing packages
import random
import numpy as np
import pandas as pd
import time
import scipy.sparse as sparse
import operator
import implicit

In [3]:
#the LogisticMF file
import random
import numpy as np
import pandas as pd
import operator
from scipy import sparse


class LogisticMF:

    def __init__(self, counts, num_factors=40, reg_param=100.0, gamma=1.0,
                 iterations=40, num_users = 6518, num_items = 4036):
        self.counts = counts
        self.num_users = num_users
        self.num_items = num_items
        self.num_factors = num_factors
        self.iterations = iterations
        self.reg_param = reg_param
        self.gamma = gamma

    def train_model(self):

        self.ones = np.ones((self.num_users, self.num_items))
        self.user_vectors = np.random.normal(size=(self.num_users,
                                                   self.num_factors))
        self.item_vectors = np.random.normal(size=(self.num_items,
                                                   self.num_factors))
        self.user_biases = np.random.normal(size=(self.num_users, 1))
        self.item_biases = np.random.normal(size=(self.num_items, 1))

        user_vec_deriv_sum = np.zeros((self.num_users, self.num_factors))
        item_vec_deriv_sum = np.zeros((self.num_items, self.num_factors))
        user_bias_deriv_sum = np.zeros((self.num_users, 1))
        item_bias_deriv_sum = np.zeros((self.num_items, 1))
        for i in range(self.iterations):
            # Fix items and solve for users
            # take step towards gradient of deriv of log likelihood
            # we take a step in positive direction because we are maximizing LL
            user_vec_deriv, user_bias_deriv = self.deriv(True)
            user_vec_deriv_sum += np.square(user_vec_deriv)
            user_bias_deriv_sum += np.square(user_bias_deriv)
            vec_step_size = self.gamma / np.sqrt(user_vec_deriv_sum)
            bias_step_size = self.gamma / np.sqrt(user_bias_deriv_sum)
            self.user_vectors += vec_step_size * user_vec_deriv
            self.user_biases += bias_step_size * user_bias_deriv

            # Fix users and solve for items
            # take step towards gradient of deriv of log likelihood
            # we take a step in positive direction because we are maximizing LL
            item_vec_deriv, item_bias_deriv = self.deriv(False)
            item_vec_deriv_sum += np.square(item_vec_deriv)
            item_bias_deriv_sum += np.square(item_bias_deriv)
            vec_step_size = self.gamma / np.sqrt(item_vec_deriv_sum)
            bias_step_size = self.gamma / np.sqrt(item_bias_deriv_sum)
            self.item_vectors += vec_step_size * item_vec_deriv
            self.item_biases += bias_step_size * item_bias_deriv

    def deriv(self, user):
        if user:
            vec_deriv = np.dot(self.counts, self.item_vectors)
            bias_deriv = np.expand_dims(np.sum(self.counts, axis=1), 1)

        else:
            vec_deriv = np.dot(self.counts.T, self.user_vectors)
            bias_deriv = np.expand_dims(np.sum(self.counts, axis=0), 1)
        A = np.dot(self.user_vectors, self.item_vectors.T)
        A += self.user_biases
        A += self.item_biases.T
        A = np.exp(A)
        A /= (A + self.ones)
        A = (self.counts + self.ones) * A

        if user:
            vec_deriv -= np.dot(A, self.item_vectors)
            bias_deriv -= np.expand_dims(np.sum(A, axis=1), 1)
            # L2 regularization
            vec_deriv -= self.reg_param * self.user_vectors
        else:
            vec_deriv -= np.dot(A.T, self.user_vectors)
            bias_deriv -= np.expand_dims(np.sum(A, axis=0), 1)
            # L2 regularization
            vec_deriv -= self.reg_param * self.item_vectors
        return (vec_deriv, bias_deriv)

    def log_likelihood(self):
        loglik = 0
        A = np.dot(self.user_vectors, self.item_vectors.T)
        A += self.user_biases
        A += self.item_biases.T
        B = A * self.counts
        loglik += np.sum(B)

        A = np.exp(A)
        A += self.ones

        A = np.log(A)
        A = (self.counts + self.ones) * A
        loglik -= np.sum(A)

        # L2 regularization
        loglik -= 0.5 * self.reg_param * np.sum(np.square(self.user_vectors))
        loglik -= 0.5 * self.reg_param * np.sum(np.square(self.item_vectors))
        return loglik

    def print_vectors(self):
        user_vecs_file = open('logmf-user-vecs-%i' % self.num_factors, 'w')
        for i in range(self.num_users):
            vec = ','.join(map(str, self.user_vectors[i]))
            line = '%i,%s\n' % (i, vec)
            user_vecs_file.write(line)
        user_vecs_file.close()
        item_vecs_file = open('logmf-item-vecs-%i' % self.num_factors, 'w')
        for i in range(self.num_items):
            vec = ','.join(map(str, self.item_vectors[i]))
            line = '%i,%s\n' % (i, vec)
            item_vecs_file.write(line)
        a = item_vecs_file
        item_vecs_file.close()
        return a






In [4]:
#Loading the matrix
def load_matrix(filepath, num_users, num_items):
    counts = np.zeros((num_users, num_items))
    for i, line in enumerate(open(filepath, 'r')):
        user, item, count = line.strip().split(',')
        user = int(user)
        item = int(item)
        count = float(count)
        counts[user][item] = count
    return counts

def sparser(originalmatrix, num_users, num_items):
    counts = sparse.dok_matrix((num_users, num_items), dtype=float)
    for i in range(len(originalmatrix)):
        row = originalmatrix[i]
        for j in range(len(row)):
            if row[j] > 0:
                user = int(i)
                item = int(j)
                count = float(row[j])
                counts[user, item] = count
    counts = counts.tocsr()
    return counts

#Please enter the path to the input data
filepath = 'C:/Users/peter/Documents/uvt/implicit/finalxdata.csv'
matrix = load_matrix(filepath, num_users = 6518, num_items = 4036)


In [5]:
# Functions for the hiding of the user-item interactions for the validation and test set
def interactiontodrop(rownumber, row):
    scoreditems = []
    items = dict(enumerate(row))
    for k in items:
        if items[k] > 0:
            scoreditems.append(k)
    random.seed(1)
    if not scoreditems:
        print(rownumber)
    return [rownumber, random.choice(scoreditems)]


def dropper(matrix, n):
    r = list(range(len(matrix)))
    random.seed(1)
    sample = random.sample(r,n)
    toodrop = []
    for x in sample:
        toodrop.append(interactiontodrop(x, matrix[x]))
    for item in toodrop:
        matrix[item[0]][item[1]] = 0
    return toodrop


In [6]:
#Creating the test set
test_set = dropper(matrix, 650)
originalmatrix = matrix


In [7]:
#Functions for the evaluation

def prediction(userid, user_vectors, item_vectors, user_biases, item_biases, originalmatrix, n=4036):
    pred = np.dot(user_vectors[userid], item_vectors.T)
    pred = pred + user_biases[userid]
    pred=pred.tolist()
    item_biases=item_biases.tolist()
    new_item_biases = [i[0] for i in item_biases]
    pred = [sum(x) for x in zip(pred, new_item_biases)]
    pred = np.asarray(pred)
    predictions = np.exp(pred)/(1+np.exp(pred))
    for i in range(4036):
        if originalmatrix[userid][i] > 0:
            predictions[i] = -6000
    dict = {key: value for (key, value) in (enumerate(predictions))}
    sorted_dict = sorted(dict.items(), key=operator.itemgetter(1), reverse=True)
    recommendation = []
    for i in range(n):
        recommendation.append(sorted_dict[i][0])
    return recommendation


def hlu_calc(dropped_items, coefficients, originalmatrix):
    model_hlus = []
    user_vectors, item_vectors, user_biases, item_biases = coefficients
    for b in dropped_items:
        ranking = (prediction(b[0], user_vectors, item_vectors, user_biases, item_biases,
                                       originalmatrix).index(b[1]) + 1)
        if ranking < 250:
            ind_hlu = 1 / (2 ** ((ranking) / 10))
            model_hlus.append(ind_hlu)
        else:
            model_hlus.append(0.0000000001)
    return model_hlus

The model building part. Only this time, my pc needs too much memory space. However, I can run the code if I copy all notebook cells directly into a python file.

In [8]:
# Building the model
alpha = 10
matrix = matrix*alpha
lambdaa = 50
t1 = time.time()
model = LogisticMF(counts=matrix, reg_param=lambdaa, num_factors=40, iterations=20, gamma = 1)
model.train_model()
print("This algorithm is not that fast, it took {} seconds to build the model".format(time.time()-t1))
coefficients = [model.user_vectors, model.item_vectors, model.user_biases, model.item_biases]

MemoryError: 

In [11]:
print(np.average(hlu_calc(test_set, coefficients, originalmatrix
                         )))

NameError: name 'coefficients' is not defined